Imports

In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

pd.set_option("display.max_columns", 200)


Connexion DB + extraction

In [2]:
DATABASE_URL = os.environ["DATABASE_URL"]  # doit exister dans ton environnement
engine = create_engine(DATABASE_URL)

QUERY = """
select
  activity_id,
  athlete_id,
  start_date,
  moving_time_s,
  distance_m,
  elevation_gain_m,
  pace_s_per_km,
  elev_m_per_km,
  avg_hr,
  max_hr,
  avg_watts,
  max_watts,
  hr_x_time,
  dist_7d_m,
  elev_7d_m,
  time_7d_s,
  dist_28d_m,
  elev_28d_m,
  time_28d_s
from activity_features
where start_date is not null
order by start_date
"""

df = pd.read_sql(QUERY, engine)
df["start_date"] = pd.to_datetime(df["start_date"], utc=True)
df.shape


(30, 19)

Feature engineering cohérent (mêmes features qu’on utilisera au modèle)

In [3]:
df_fe = df.copy()

# densité de D+ (m/m) et par km
df_fe["elev_density_m_per_m"] = df_fe["elevation_gain_m"] / df_fe["distance_m"]
df_fe["elev_density_m_per_km"] = df_fe["elevation_gain_m"] / (df_fe["distance_m"] / 1000.0)

# ratios de charge (attention aux 0 / NaN)
df_fe["charge_ratio_dist_7_28"] = df_fe["dist_7d_m"] / df_fe["dist_28d_m"]
df_fe["charge_ratio_elev_7_28"] = df_fe["elev_7d_m"] / df_fe["elev_28d_m"]
df_fe["charge_ratio_time_7_28"] = df_fe["time_7d_s"] / df_fe["time_28d_s"]

# logs (souvent utile avec distributions très skew)
df_fe["log_distance_m"] = np.log1p(df_fe["distance_m"])
df_fe["log_elev_gain_m"] = np.log1p(df_fe["elevation_gain_m"])

# nettoyage inf / -inf
df_fe = df_fe.replace([np.inf, -np.inf], np.nan)

df_fe[["distance_m","elevation_gain_m","elev_density_m_per_km","charge_ratio_dist_7_28"]].describe().T


,count,mean,std,min,25%,50%,75%,max
distance_m,30.0,4954.650000,2676.947221,0.00000,3089.475000,4642.90000,6126.350000,10423.000000
elevation_gain_m,30.0,28.863333,27.667664,0.00000,11.475000,25.25000,35.575000,135.400000
elev_density_m_per_km,29.0,6.348322,5.635957,0.00000,2.487761,4.95408,9.135223,26.325511
charge_ratio_dist_7_28,30.0,0.767506,0.276774,0.13418,0.557926,1.00000,1.000000,1.000000


Définir X / y + filtrer les lignes utilisables

In [4]:
TARGET = "pace_s_per_km"

FEATURES = [
    "distance_m",
    "elevation_gain_m",
    "elev_density_m_per_km",
    "dist_7d_m",
    "elev_7d_m",
    "time_7d_s",
    "dist_28d_m",
    "elev_28d_m",
    "time_28d_s",
    "charge_ratio_dist_7_28",
    "charge_ratio_elev_7_28",
    "charge_ratio_time_7_28",
    "log_distance_m",
    "log_elev_gain_m",
    # optionnel: cardio si dispo
    #"avg_hr",
    #"avg_watts",
]

# Garde uniquement ce qui est utilisable
df_model = df_fe.dropna(subset=[TARGET]).copy()

# (Optionnel) : si tu veux un modèle sans HR/watts, commente les deux features ci-dessus
df_model = df_model.dropna(subset=[c for c in FEATURES if c in df_model.columns])

df_model = df_model.sort_values("start_date").reset_index(drop=True)

print("Rows for modeling:", df_model.shape)
df_model[[TARGET] + [c for c in FEATURES if c in df_model.columns]].head()


Rows for modeling: (28, 26)


,pace_s_per_km,distance_m,elevation_gain_m,elev_density_m_per_km,dist_7d_m,elev_7d_m,time_7d_s,dist_28d_m,elev_28d_m,time_28d_s,charge_ratio_dist_7_28,charge_ratio_elev_7_28,charge_ratio_time_7_28,log_distance_m,log_elev_gain_m
0,390.895026,6075.8,30.1,4.954080,6075.8,30.1,2375.0,6075.8,30.1,2375.0,1.000000,1.000000,1.000000,8.712234,3.437208
1,785.178061,3743.1,54.9,14.666987,9818.9,85.0,5314.0,9818.9,85.0,5314.0,1.000000,1.000000,1.000000,8.227937,4.023564
2,730.610471,4603.0,13.8,2.998045,4603.0,13.8,3363.0,4603.0,13.8,3363.0,1.000000,1.000000,1.000000,8.434681,2.694627
3,356.500899,2336.6,0.0,0.000000,6939.6,13.8,4196.0,6939.6,13.8,4196.0,1.000000,1.000000,1.000000,7.756880,0.000000
4,394.924478,3436.1,4.0,1.164110,5772.7,4.0,2190.0,10375.7,17.8,5553.0,0.556367,0.224719,0.394381,8.142383,1.609438


In [6]:
df_model.head()

,activity_id,athlete_id,start_date,moving_time_s,distance_m,elevation_gain_m,pace_s_per_km,elev_m_per_km,avg_hr,max_hr,avg_watts,max_watts,hr_x_time,dist_7d_m,elev_7d_m,time_7d_s,dist_28d_m,elev_28d_m,time_28d_s,elev_density_m_per_m,elev_density_m_per_km,charge_ratio_dist_7_28,charge_ratio_elev_7_28,charge_ratio_time_7_28,log_distance_m,log_elev_gain_m
0,4322970997,71669830,2020-11-11 15:05:09+00:00,2375,6075.8,30.1,390.895026,4.954080,None,None,NaN,None,None,6075.8,30.1,2375.0,6075.8,30.1,2375.0,0.004954,4.954080,1.000000,1.000000,1.000000,8.712234,3.437208
1,4337168133,71669830,2020-11-14 15:17:41+00:00,2939,3743.1,54.9,785.178061,14.666987,None,None,NaN,None,None,9818.9,85.0,5314.0,9818.9,85.0,5314.0,0.014667,14.666987,1.000000,1.000000,1.000000,8.227937,4.023564
2,4677873862,71669830,2021-01-25 09:01:38+00:00,3363,4603.0,13.8,730.610471,2.998045,None,None,NaN,None,None,4603.0,13.8,3363.0,4603.0,13.8,3363.0,0.002998,2.998045,1.000000,1.000000,1.000000,8.434681,2.694627
3,4684305941,71669830,2021-01-26 14:29:21+00:00,833,2336.6,0.0,356.500899,0.000000,None,None,NaN,None,None,6939.6,13.8,4196.0,6939.6,13.8,4196.0,0.000000,0.000000,1.000000,1.000000,1.000000,7.756880,0.000000
4,4717359618,71669830,2021-02-01 16:05:41+00:00,1357,3436.1,4.0,394.924478,1.164110,None,None,NaN,None,None,5772.7,4.0,2190.0,10375.7,17.8,5553.0,0.001164,1.164110,0.556367,0.224719,0.394381,8.142383,1.609438


Sauvegarde du dataset final (c’est CE fichier qu’on utilisera dans Modeling)

In [5]:
os.makedirs("data/processed", exist_ok=True)
df_model.to_parquet("data/processed/dataset_model.parquet", index=False)
print("Saved: data/processed/dataset_model.parquet", df_model.shape)

Saved: data/processed/dataset_model.parquet (28, 26)
